In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df = spark.read.format("parquet")\
                .load("/Volumes/db_dlt_proj/ext_location/bronze_external_location/products/")
df = df.drop("_rescued_data")

### Transformations

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION db_dlt_proj.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
RETURN (p_price * 0.90);

In [0]:
%sql
-- select product_id, price, db_dlt_proj.bronze.discount_func(price) as discounted_price from products

In [0]:
df = df.withColumn('discounted_price', expr("db_dlt_proj.bronze.discount_func(price)"))
display(df)

## creating functio using Python

In [0]:
%sql
CREATE OR REPLACE FUNCTION db_dlt_proj.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

Writing data into delta table

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
            .save("/Volumes/db_dlt_proj/ext_location/silver_external_location/products/")

df.write.format("delta")\
        .mode("overwrite")\
          .saveAsTable("db_dlt_proj.silver.products")